In [1]:
import pandas as pd
import numpy as np
import pathlib
from dataset_loader import load_dataframes
import os

In [2]:
base_dir = str(pathlib.Path().resolve())
in_path = f'{base_dir}/new_edges/new_ddc.csv'
df = pd.read_csv(in_path)
#############################################################
drug_pair_dict = dict()
for row in df.values:
    if row[2] == '-':
        pass
    else:
        pair = f'{row[4]} - {row[5]}'
        pair2 = f'{row[5]} - {row[4]}' 
        if pair in list(drug_pair_dict.keys()) or pair2 in list(drug_pair_dict.keys()):
            try:
                count = drug_pair_dict[pair]
                count += 1
                drug_pair_dict[pair] = count
            except Exception as e:
                count = drug_pair_dict[pair2]
                count += 1
                drug_pair_dict[pair2] = count
        else:
            drug_pair_dict[pair] = 1
#############################################################
d1_list = []
d2_list = []
pair_count = []
for key in list(drug_pair_dict.keys()):
    ds = key.split(' - ')
    d1_list.append(ds[0])
    d2_list.append(ds[1])
    pair_count.append(drug_pair_dict[key])
#############################################################
if 'new_edges' not in os.listdir(f'{base_dir}/'):
    os.mkdir( f'{base_dir}/new_edges/')
#############################################################
new_df = pd.DataFrame()
new_df['d1'] = d1_list
new_df['d2'] = d2_list
new_df['count'] = pair_count
new_df.sort_values('count', ascending=False, inplace=True)
new_df.to_csv(f'{base_dir}/new_edges/new_ddc_unique.csv', index=False)

In [38]:
new_df

,d1,d2,count
14,Lamivudine,Delavirdine,8
11,Lamivudine,Rifapentine,8
10,Lamivudine,Indinavir,7
18,Lamivudine,Darunavir,7
12,Lamivudine,Rifabutin,6
...,...,...,...
267,Valproic,Carmustine,1
266,Aspirin,Furosemide,1
265,Aspirin,Ketoprofen,1
263,Aspirin,Ergotamine,1


# loading all edges with drug names (not ddc codes)

In [20]:
final_dt_df, final_w2v_df, final_nv_df, final_fin_df, final_in_df, final_se_df, all_df = load_dataframes()

out_dir = str(pathlib.Path().resolve())
out_path = f'{out_dir}/datasets/dcdb.csv'

if 'dcdb.csv' not in os.listdir(f'{out_dir}/datasets/'):
    src_path = f'{out_dir}/datasets/COMPONENTS.txt'
    f = open(src_path, 'r')

    csv_1 = []
    csv_2 = []

    for line in f.readlines():
        each = line.split(' ')[0].split('\t')
        csv_1.append(each[0])
        csv_2.append(each[1])

    print(len(csv_1))
    print(len(csv_2))

    csv_1 = np.array(csv_1[1:])
    csv_2 = np.array(csv_2[1:])

    csv_1 = np.reshape(csv_1, (len(csv_1), 1))
    csv_2 = np.reshape(csv_2, (len(csv_2), 1))

    drug_df = pd.DataFrame(np.concatenate([csv_1, csv_2], axis=1), columns=['drug', 'name'])
    drug_df

    drug_df.to_csv(out_path, index=False)
else:
    drug_df = pd.read_csv(out_path)

drug_names = dict()

for drug in drug_df.values:
    drug = list(drug)
    drug_names[drug[0]] = drug[1]

all_edges = []

for edge in all_df.values:
    if list(edge) not in all_edges and [edge[1], edge[0]] not in all_edges:
        all_edges.append(list(edge))
        all_edges.append([edge[1], edge[0]])

d1_list = []
d2_list = []
t_list = []
for edge in all_edges:
    d1 = drug_names[edge[0]]
    d2 = drug_names[edge[1]]

    e1 = f'{d1}-{d2}'
    e2 = f'{d2}-{d1}'

    if e1 in t_list or e2 in t_list:
        pass
    else:
        t_list.append(e1)
        d1_list.append(d1)
        d2_list.append(d2)

df_all_names = pd.DataFrame()
df_all_names['d1'] = d1_list
df_all_names['d2'] = d2_list
df_all_names.to_csv(f'{out_dir}/all_edges_names.csv', index=False)